In [1]:
####################################################################
#######   Import the required libraries according to the given model
####################################################################
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from utilities3 import *   #Import the utilities3 according to your requirement
import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from adam import Adam   #####change the Adam according to the required model
import math
import os
torch.manual_seed(0)
np.random.seed(0)

ModuleNotFoundError: No module named 'torch'

In [ ]:
################################################
######## Importing the things for making figures
################################################

import scipy 
from scipy import io
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from new_plot import *
import pandas as pd
from os import listdir
from os.path import isfile, join
import os
set_things()
set_font(size=15, family='Arial', weight='normal')
matplotlib.rcParams['figure.figsize'] = 5.20, 5.20
matplotlib.rcParams['lines.linewidth'] = 2.5
%matplotlib inline
import seaborn as sns
import matplotlib.ticker as tck
import matplotlib.gridspec as gridspec
import geopandas as gpd
import matplotlib.cm as cm
from shapely.geometry import Polygon
#import cartopy.crs as ccrs
from scipy.interpolate import griddata
from datetime import datetime, timedelta
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

In [ ]:
###############################
###### Define the Models here
###############################

class SpectralConv2d_FNO(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d_FNO, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  # Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-2), x.size(-1) // 2 + 1, dtype=torch.cfloat,
                             device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        # Return to physical space
        x = torch.fft.irfft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x

class FNO2d(nn.Module):
    def __init__(self, in_dim = 10, out_dim = 1, num_basis = 12, d_model = 20):
        super(FNO2d, self).__init__()
        self.in_channels = in_dim
        self.out_channels = out_dim
        self.modes1 = num_basis
        self.modes2 = num_basis
        self.width = d_model
        self.padding = [int(x) for x in [0, 0]]

        self.conv0 = SpectralConv2d_FNO(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d_FNO(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d_FNO(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d_FNO(self.width, self.width, self.modes1, self.modes2)
        self.w0 = nn.Conv2d(self.width, self.width, 1)
        self.w1 = nn.Conv2d(self.width, self.width, 1)
        self.w2 = nn.Conv2d(self.width, self.width, 1)
        self.w3 = nn.Conv2d(self.width, self.width, 1)

        self.fc0 = nn.Linear(self.in_channels + 2, self.width)  # input channel is 3: (a(x, y), x, y)
        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, self.out_channels)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)
        if not all(item == 0 for item in self.padding):
            x = F.pad(x, [0, self.padding[0], 0, self.padding[1]])

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        if not all(item == 0 for item in self.padding):
            x = x[..., :-self.padding[1], :-self.padding[0]]
        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)

In [ ]:
####################################
##### Complex FFNO
####################################
import torch.nn.functional as F
import torch.nn as nn
import torch
import numpy as np
import math
from math import pi

import torch

# Complex Pytorch
import torchcomplex.nn as cn

# Fractional Fourier Transform 
from torch_frft.layer import DFrFTLayer, FrFTLayer
from torch_frft.frft_module import frft
from torch_frft.dfrft_module import dfrft, dfrftmtx

################################################################
# complex GeLU
################################################################
class ComplexGELU(nn.Module):
    def __init__(self):
        super(ComplexGELU, self).__init__()

    def forward(self, input):
        real_gelu = F.gelu(input.real)
        imag_gelu = F.gelu(input.imag)

        return torch.complex(real_gelu, imag_gelu)

################################################################
# complex SELU
################################################################
class ComplexSELU(nn.Module):
    def __init__(self):
        super(ComplexSELU, self).__init__()

    def forward(self, input):
        real_selu = F.selu(input.real)
        imag_selu = F.selu(input.imag)

        return torch.complex(real_selu, imag_selu)
    
################################################################
# complex fourier layer
################################################################
class SpectralConv2d_CONO(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d_CONO, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  # Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.fft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-2), x.size(-1), dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        # Return to physical space
        x = torch.fft.ifft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x
    
################################################################
# complex fractional fourier layer
################################################################

class SpectralFractionalConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2, alpha: float = 0.5):
        super(SpectralFractionalConv2d, self).__init__()

        """
        2D Complex Fractional Fourier layer. It does FFFT, linear transform, and Inverse FFFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        # self.modes1 = modes1  # Number of Fourier modes to multiply, at most floor(N/2) + 1
        # self.modes2 = modes2
        
        self.alpha1 = nn.Parameter(torch.tensor(alpha, dtype=torch.float32), requires_grad=True)
        self.alpha2 = nn.Parameter(torch.tensor(alpha, dtype=torch.float32), requires_grad=True)

        #self.alpha1 = alpha
        #self.alpha2 = alpha
        
        self.layer = cn.Conv2d(self.in_channels, self.out_channels, 1)

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # Compute Fractional Fourier coeffcients up to factor of e^(- something constant)
        x_ft = dfrft(dfrft(x, self.alpha1, dim=-1), self.alpha2, dim=-2)
        
        # 1x1 convolutionl in complex domain
        out_ft = self.layer(x_ft)

        # Return to physical space
        x = dfrft(dfrft(out_ft, -self.alpha2, dim=-2), -self.alpha1, dim=-1)
        return x
    
################################################################
# Real to Complex Conversion
################################################################
def real_to_complex(x, width):
    x_r = x[:, :width, :, :]
    x_i = x[:, width:, :, :]
    x_complex = torch.complex(x_r, x_i)

    return x_complex

################################################################
# Complex to Real Conversion
################################################################
def complex_to_real(x):
    # Ensure that the input tensor is complex
    if not torch.is_complex(x):
        raise ValueError("Input tensor must be complex.")

    # Separate real and imaginary parts
    x_r = x.real
    x_i = x.imag

    # Concatenate real and imaginary parts along the specified dimension
    result = torch.cat((x_r, x_i), dim=-3)

    return result

################################################################
# Learning Imaginary Part
################################################################
class ComplexBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ComplexBlock, self).__init__()
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.GELU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        )

    def forward(self, x):
        return self.block(x)
    
################################################################
# Complex Channel Mixing
################################################################
class ComplexChannelMixing(cn.Module):
    def __init__(self, in_channels, out_channels):
        super(ComplexChannelMixing, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.act = ComplexGELU()
        self.linear = cn.Linear(self.in_channels, self.out_channels)
        # self.norm = cn.BatchNorm2d(in_channels)
        
    def forward(self, x):
        x = self.act(x)
        x = x.permute(0, 2, 3, 1)
        x = self.linear(x)
        x = x.permute(0, 3, 1, 2)
        
        return x

################################################################
# Complex block
################################################################
class ConvBNActivation(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 1, stride=1, padding=0, activation=ComplexGELU()):
        super(ConvBNActivation, self).__init__()
        
        self.conv = cn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.bn = cn.BatchNorm2d(out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

class CONO(nn.Module):
    def __init__(self, in_dim = 10, out_dim = 1, num_basis = 12, d_model = 20):
        super(CONO, self).__init__()
        in_channels = in_dim
        out_channels = out_dim
        self.modes1 = num_basis
        self.modes2 = num_basis
        self.width = d_model
        self.act = ComplexGELU()
        
        self.padding = [int(x) for x in [0, 0]]

        self.conv0 = SpectralConv2d_CONO(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d_CONO(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d_CONO(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d_CONO(self.width, self.width, self.modes1, self.modes2)
        
        self.fconv0 = SpectralFractionalConv2d(self.width, self.width, self.modes1, self.modes2)
        self.fconv1 = SpectralFractionalConv2d(self.width, self.width, self.modes1, self.modes2)
        self.fconv2 = SpectralFractionalConv2d(self.width, self.width, self.modes1, self.modes2)
        self.fconv3 = SpectralFractionalConv2d(self.width, self.width, self.modes1, self.modes2)
        
        self.w0 = cn.Conv2d(self.width, self.width, 1)
        self.w1 = cn.Conv2d(self.width, self.width, 1)
        self.w2 = cn.Conv2d(self.width, self.width, 1)
        self.w3 = cn.Conv2d(self.width, self.width, 1)

        self.fc0 = nn.Linear(in_channels + 2, self.width)  # input channel is 3: (a(x, y), x, y)
        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, out_channels)
        
        self.residual = ConvBNActivation(in_channels=self.width, out_channels=self.width)
        self.complexblock = ComplexBlock(in_channels=self.width, out_channels=self.width)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)
        
        if not all(item == 0 for item in self.padding):
            x = F.pad(x, [0, self.padding[0], 0, self.padding[1]])

        # Real to complex conversion
        x_r = x
        x_i = self.complexblock(x)
        x = torch.complex(x_r, x_i)
        x = self.residual(x)
        
        x1 = self.conv0(x)
        x2 = self.w0(x)
        x3 = self.fconv0(x)
        x = x1 + x2 + x3
        # x = x1 + x2
        x = self.act(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x3 = self.fconv1(x)
        x = x1 + x2 + x3
        # x = x1 + x2
        x = self.act(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x3 = self.fconv2(x)
        x = x1 + x2 + x3
        # x = x1 + x2
        x = self.act(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x3 = self.fconv3(x)
        x = x1 + x2 + x3
        # x = x1 + x2
        
        # Complex to Real Conversion
        x = x.real
        # x = complex_to_real(x)

        if not all(item == 0 for item in self.padding):
            x = x[..., :-self.padding[1], :-self.padding[0]]
        
        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)


In [ ]:
##################################
##### Reading Test Data as whole
##################################

def read_test(T_in, reader1, reader2, T):
    test_a = reader1.read_field('a')[:, :T_in, :, :]
    test_u = reader1.read_field('a')[:, T_in:T + T_in, :, :]
    test_a = torch.cat((test_a, reader2.read_field('a')[:, :T_in, :, :]), 0)
    test_u = torch.cat((test_u, reader2.read_field('a')[:, T_in:T_in + T, :, :]), 0)
    test_a = test_a.permute(0, 2, 3, 1)
    test_u = test_u.permute(0, 2, 3, 1)
    print(test_a.shape)
    return test_a, test_u

####################################
####### Testing Loop for the models
####################################

def testing_loop(model, test_loader, T, test_u):
    prediction = torch.zeros(test_u.shape)
    model.eval()
    index = 0
    with torch.no_grad():
        for xx, yy in test_loader:
            xx = xx.to(device)
            yy = yy.to(device)

            for t in range(0, T, step):
                im = model(xx)
                if t == 0:
                    pred = im
                else:
                    pred = torch.cat((pred, im), -1)

                xx = torch.cat((xx[..., step:], im), dim=-1)
            prediction[index] = pred
            index = index + 1
            if index % 100 == 0:
                print(index)
    return prediction

In [ ]:
#######################################
######   Change the hyperparameters
#######################################

ntrain = 3000
ntest = 500
modes = 12
width = 20

epochs = 500
learning_rate = 0.001
scheduler_step = 50
scheduler_gamma = 0.5
sub = 1
S1 = 140
S2 = 124
step = 1
print(epochs, learning_rate, scheduler_step, scheduler_gamma)

In [ ]:
T_in = 10
T1 = 16
T2 = 20
T3 = 72
modeltype = ['fno', 'cono']
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
abs_max = #
abs_min = #

In [ ]:
##########################################
###### Data Path for whole test data
##########################################

TEST_PATH1 = 'outsample1.mat'
TEST_PATH2 = 'outsample2.mat'
reader1 = MatReader(TEST_PATH1)
reader2 = MatReader(TEST_PATH2)
test_a, test_u = read_test(T_in, reader1, reader2, T2)
print(test_a.shape, test_u.shape)

################################################
#######   Load the data for 72 hours outsample
################################################

TEST_PATH3 = 'outsample_72hours.mat'
reader3 = MatReader(TEST_PATH3)
test_a_72 = reader3.read_field('a')[:, :T_in, :, :]
test_u_72 = reader3.read_field('a')[:, T_in: T_in + T3, :, :]
test_a_72 = test_a_72.permute(0, 2, 3, 1)
test_u_72 = test_u_72.permute(0, 2, 3, 1)
print(test_a_72.shape, test_u_72.shape)

####################################
#####    Load the maximum Dataset
####################################
TEST_PATH4 = 'Data_Max_Sum.mat'
reader4 = MatReader(TEST_PATH4)
test_a_m = reader4.read_field('a')[:, 0:T_in, :, :]
test_u_m = reader4.read_field('a')[:, T_in:T_in+24, :, :]
test_a_m = (test_a_m - abs_min)/(abs_max - abs_min)
test_u_m = (test_u_m - abs_min)/(abs_max - abs_min)
test_a_m = test_a_m.permute(0, 2, 3, 1)
test_u_m = test_u_m.permute(0, 2, 3, 1)

In [ ]:
########################
#### Load Validation dataset
########################
TEST_PATH = "test_data.mat"
reader5 = MatReader(TEST_PATH)
test_a_t = reader5.read_field('a')[:, :T_in, :, :]
test_u_t = reader5.read_field('a')[:, T_in:, :, :]
test_a_t = test_a_t.permute(0, 2, 3, 1)
test_u_t = test_u_t.permute(0, 2, 3, 1)
print(test_a_t.shape, test_u_t.shape)

In [ ]:
###########################################################
#########  folder and the locations for storing the results
###########################################################

mainpath = 'final_simulations_myhpc/'
variations = ['_10_4/', '_10_6/','_10_16/']
lat = [93, 50, 25, 68, 82, 101, 69]     # for 7 grids
long = [38, 21, 40, 20, 98, 42, 76]     # for 7 grids

t_stamp = pd.read_csv('timestamp_cities.csv')
time_72 = pd.read_csv('outsample_time.csv')        #Change the address of the files
del time_72['Unnamed: 0']
time_72['0'] = pd.to_datetime(time_72['0'], dayfirst = True)
shapefile = gpd.read_file('India_Boundary/India_Boundary.shp')
data_time = scipy.io.loadmat('Data_Max_Sum.mat')['time']

In [ ]:
extent_mat = (67.85, 97.93486, 7.060257, 38.004227)

In [ ]:
###### Inferencing for all the datasets


for m in modeltype:
    subfolder = m.upper()
    for variation in variations:

        print(torch.max(test_a_t), torch.max(test_u_t), torch.min(test_a_t), torch.min(test_u_t))
        if m == 'fno':
            model = FNO2d()
            modelpath = 'fno_2D_channels_20' + variation[:-1] + '.pt'
        else:
            model = CONO()
            modelpath = 'complex_ffno_2D' + variation[:-1] + '.pt'
        myloss = LpLoss(size_average=False)
        path = mainpath + subfolder + variation + modelpath
        print(path)
        print(model.state_dict()['conv0.weights1'][0, 0, 0, 0 ])
        model.load_state_dict(torch.load(path))
        print(model.state_dict()['conv0.weights1'][0, 0, 0, 0 ])
        model = model.to('cuda')
        model.eval()
        print(count_params(model))
        myloss = LpLoss(size_average=False)
        
        ################################
        ### Testing validtion for 16 timesteps
        ################################
        test_loader1 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a_t, test_u_t), batch_size=1, shuffle=False)
        prediction1 = testing_loop(model, test_loader1, T1, test_u_t)
        print(torch.min(prediction1), torch.max(prediction1))
        scipy.io.savemat(mainpath + subfolder + variation + 'prediction_test_16_steps_'+ m + str(T_in) + '_' + variation.split('_')[2][:-1] +'.mat', 
                         mdict={'pred': prediction1.cpu().numpy(), 'test_u': test_u_t.cpu().numpy()})

        ################################
        ### Testing outsample for 20 timesteps
        ################################
        test_loader2 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=1, shuffle=False)
        prediction2 = testing_loop(model, test_loader2, T2, test_u)
        print(torch.min(prediction2), torch.max(prediction2))
        scipy.io.savemat(mainpath + subfolder + variation + 'prediction_outsample_20_steps_'+ m + str(T_in) + '_' + variation.split('_')[2][:-1] +'.mat', 
                         mdict={'pred': prediction2.cpu().numpy(), 'test_u': test_u.cpu().numpy()})
        
        ##################################
        # Testing outsample for 72 timesteps
        ##################################
        
        test_loader3 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a_72, test_u_72), batch_size=1, shuffle=False)
        prediction3 = testing_loop(model, test_loader3, T3, test_u_72)
        print(torch.min(prediction3), torch.max(prediction3))
        scipy.io.savemat(mainpath + subfolder + variation + 'prediction_72_'+ m + str(T_in) + '_' + variation.split('_')[2][:-1] +'.mat', 
                         mdict={'pred': prediction3.cpu().numpy(), 'test_u': test_u_72.cpu().numpy()})

        ##########################
        # Maximum dataset testing
        ##########################
        test_loader4 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a_m, test_u_m), batch_size=1, shuffle=False)
        prediction4 = testing_loop(model, test_loader4, 24, test_u_m)
        scipy.io.savemat(mainpath + subfolder + variation + 'prediction_max_'+ m + str(T_in) + '_' + variation.split('_')[2][:-1] +'.mat', 
                         mdict={'pred': prediction4.cpu().numpy(), 'test_u': test_u_m.cpu().numpy()})

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
#### Extract city data for the first hour forecasts
def extract_cities(test_u, prediction_o1, abs_min, abs_max, mainpath, subfolder, variation, lat, long):
    for j in range(7):
        if j == 0:
            test_c = test_u[:, lat[j], long[j], 0].reshape(-1, 1)
            prediction_c = prediction_o1[:, lat[j], long[j], 0].reshape(-1, 1)
        else:
            im1 = test_u[:, lat[j], long[j], 0].reshape(-1, 1)
            im2 = prediction_o1[:, lat[j], long[j], 0].reshape(-1, 1)
            test_c = np.concatenate([test_c, im1], axis = 1)
            prediction_c = np.concatenate([prediction_c, im2], axis = 1)
    test_c = test_c * (abs_max - abs_min) + abs_min
    prediction_c = prediction_c * (abs_max - abs_min) + abs_min
    test_c = np.concatenate([test_c, prediction_c], axis = 1)
    df = pd.DataFrame(test_c, columns = cities)
    df.index = list(t_stamp['Unnamed: 0'])
    df.to_csv(mainpath + subfolder + variation + 'cities_1hour.csv')

In [ ]:
###########################
##### Code to extract the sheets from prediction on outsample data
###########################
for m in modeltype:
    subfolder = m.upper()
    cities = ['Delhi_wrf', 'Mumbai_wrf', 'Bengaluru_wrf', 
          'Ahemdabad_wrf', "Guwahati_wrf", 'Dehradun_wrf', 'Max_wrf', 
          'Delhi_' + m, 'Mumbai_' + m, 'Bengaluru_' + m, 
          'Ahemdabad_' + m, "Guwahati_" + m, 'Dehradun_' + m, 'Max_' + m]
    for variation in variations:
        print(mainpath + subfolder + variation)
        data = scipy.io.loadmat(mainpath + subfolder + variation + 'prediction_outsample_20_steps_'+ m + str(T_in) + '_' + variation.split('_')[2][:-1] +'.mat')
        prediction_o1 = data['pred']
        test_total = data['test_u']
        extract_cities(test_total, prediction_o1, abs_min, abs_max, mainpath, subfolder, variation, lat, long)